#### Loading the libraries

In [1]:
from utils.utils import get_root_project
from mlflow.models.signature import infer_signature
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import mlflow
import os

In [2]:
tracking_server = get_root_project() / 'MLFlow' / 'mlruns'
tracking_server

PosixPath('/Users/sachintripathi/Documents/Py_files/EXL_MLOps/MLFlow/mlruns')

In [3]:
os.makedirs(tracking_server, exist_ok = True)
mlflow.set_tracking_uri(tracking_server.as_uri())

In [4]:
experiment_name = "comparing_models"
try: 
    mlflow.create_experiment(name = experiment_name)
except Exception as e:
    print(e)

mlflow.set_experiment(experiment_name = experiment_name)

<Experiment: artifact_location='file:///Users/sachintripathi/Documents/Py_files/EXL_MLOps/MLFlow/mlruns/914191864866377034', creation_time=1734078793362, experiment_id='914191864866377034', last_update_time=1734078793362, lifecycle_stage='active', name='comparing_models', tags={}>

#### Generate data

In [5]:
x, y = make_classification(n_samples = 1000, n_features = 5, n_classes = 2, random_state = 42)

x_df = pd.DataFrame(x, columns = [f"feature_{i+1}" for i in range(x.shape[1])])
y_df = pd.DataFrame(y, columns = ['target'])
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size = 0.2, random_state = 42)

model_signature = infer_signature(x_train, y_train)

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


#### Training models (Baseline(DecisionTreeClassifier) and Candidate Model(RandomForestClassifier))

In [25]:
baseline_model = DecisionTreeClassifier(random_state = 42)
baseline_model.fit(x_train, y_train)

# logging the baseline model
with mlflow.start_run(run_name = "baseline") as baseline_run: 
    mlflow.log_params(baseline_model.get_params())
    artifact_path = "baseline_model"
    mlflow.sklearn.log_model(baseline_model, artifact_path, signature = model_signature)

In [26]:
# printing the URI of the baseline model
baseline_uri = f"runs:/{baseline_run.info.run_id}/{artifact_path}"
baseline_uri

'runs:/06f68e2a838548439ea8883188c9806f/baseline_model'

In [ ]:
candidate = RandomForestClassifier(random_state = 42)
candidate.fit(x_train, y_train)

candidate_predictions = candidate.predict(x_test)

# logging the candidate model
with mlflow.start_run(run_name = "candidate") as candidate_run: 
    mlflow.log_params(candidate.get_params())
    artifact_path = "candidate_model"
    mlflow.sklearn.log_model(candidate, artifact_path, signature = model_signature)
    
candidate_uri = f"runs:/{candidate_run.info.run_id}/{artifact_path}"

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [11]:
# wrapping the models
def baseline_model_func(model_input):
    return baseline_model.predict(model_input)

def candidate_model_func(model_input):
    return candidate.predict(model_input)

#### Comparison

In [12]:
eval_data_for_model = x_test.copy()
eval_data_for_model["target"] = y_test

In [13]:
# creating validation thresholds
from mlflow.models import MetricThreshold

f1_score_th = {
    "f1_score": MetricThreshold(
        threshold = 0.8,
        min_absolute_change = 0.01, 
        min_relative_change = 0.01, 
        greater_is_better = True
    )
}

In [27]:
with mlflow.start_run(run_name = "comparing models") as run: 
    results = mlflow.evaluate(
        model = candidate_model_func, 
        baseline_model = baseline_uri, 
        data = eval_data_for_model, 
        targets = "target", 
        model_type = "classifier", 
        validation_thresholds = f1_score_th
    )

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/13 14:28:01 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify

In [28]:
from mlflow.metrics import make_metric
from sklearn.metrics import f1_score

def custom_accuracy(df, __builtin_metrics):
    targets = df["target"]
    predictions = df["prediction"]
    return sum(targets == predictions)/len(targets)

def custom_f1_score(df, __builtin_metrics):
    targets = df["target"]
    predictions = df["prediction"]
    return f1_score(targets, predictions, average = "weighted")

custom_metric_accuracy = make_metric(
    eval_fn = custom_accuracy, 
    name = "custom_accuracy", 
    greater_is_better = True
)

custom_metric_f1_score = make_metric(
    eval_fn = custom_f1_score, 
    name = "custom_f1_score", 
    greater_is_better = True
)

In [29]:
with mlflow.start_run(run_name = "comparing_models") as run: 
    results = mlflow.evaluate(
        model = candidate_model_func, 
        baseline_model = baseline_uri, 
        data = eval_data_for_model,
        targets = "target", 
        model_type = "classifier", 
        validation_thresholds = f1_score_th, 
        extra_metrics = [custom_metric_accuracy, custom_metric_f1_score]
)

/Users/sachintripathi/Documents/Py_files/dev/lib/python3.12/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/12/13 14:38:14 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify